<a href="https://colab.research.google.com/github/rprimi/b5_nlp/blob/main/m2_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
install.packages("tensorflow")
library(tensorflow)
install.packages("keras")
library(keras)




install_tensorflow()

library(reticulate)
use_python("/usr/bin/python3")


install_tensorflow(
  version = "default",
  method = c("auto"),
  cuda = TRUE,
 envname = "NULL" )

install_tensorflow(version = "default", cuda = TRUE)

install.packages("devtools")
devtools::install_github("rstudio/reticulate")
install.packages("tensorflow")
library(reticulate)
use_python("/usr/local/lib/python3.10")

dist-packages
library(tensorflow)
tf_config()

py_run_string("import tensorflow as tf")
py_run_string("print(tf.__version__)")


pacakges <- c("tidyverse", "keras", "psych", "readr", "zip")
install.packages(pacakges)
detach("package:tensorflow")
detach


ERROR: ignored

In [5]:
library(tidyverse)
library(keras)
library(psych)
library(readr)
library(zip)

# library(tensorflow)


In [2]:
load("base_bfi_nlp.RData")

In [25]:
samples <- db_textos_splited$texto_dividido

tokenizer <- text_tokenizer(
    lower = TRUE )  %>%
    fit_text_tokenizer(samples)

word_index <- tokenizer$word_index

sequences <- texts_to_sequences(tokenizer, samples)

cat("Found", length(word_index), "unique tokens.\n")


List of 22
 $ python              : chr "/root/.virtualenvs/r-tensorflow/bin/python"
 $ libpython           : chr "/usr/lib/python3.10/config-3.10-x86_64-linux-gnu/libpython3.10.so"
 $ pythonhome          : chr "/root/.virtualenvs/r-tensorflow:/root/.virtualenvs/r-tensorflow"
 $ pythonpath          : chr "/usr/local/lib/R/site-library/reticulate/config:/env/python:/usr/lib/python310.zip:/usr/lib/python3.10:/usr/lib"| __truncated__
 $ prefix              : chr "/root/.virtualenvs/r-tensorflow"
 $ exec_prefix         : chr "/root/.virtualenvs/r-tensorflow"
 $ base_exec_prefix    : chr "/usr"
 $ virtualenv          : chr "/root/.virtualenvs/r-tensorflow"
 $ virtualenv_activate : chr ""
 $ executable          : chr "/root/.virtualenvs/r-tensorflow/bin/python"
 $ base_executable     : chr "/root/.virtualenvs/r-tensorflow/bin/python"
 $ version_string      : chr "3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]"
 $ version             : chr "3.10"
 $ architecture        : chr "64bit"
 $ an

ERROR: ignored

In [ ]:
load("/content/vocabulary.RData")
names(vocabulary)

# Function adapted from Chalot book
  prepare_embedding_matrix <- function(num_words, EMBEDDING_DIM, word_index) {
      MAX_NUM_WORDS = num_words
      embedding_matrix <- matrix(0L, nrow = num_words+1, ncol = EMBEDDING_DIM)
      for (word in names(word_index)) {
        index <- word_index[[word]]
        if (index >= MAX_NUM_WORDS)
          next
        embedding_vector <- as.numeric(vocabulary[vocabulary$word == word, 3:602])
        if (!is.null(embedding_vector)) {
          # words not found in embedding index will be all-zeros.
          embedding_matrix[index+1,] <- embedding_vector
        }
      }
      embedding_matrix
    }

# Creates embedding matrix
     embedding_matrix <- prepare_embedding_matrix(
         num_words = 66128,
         EMBEDDING_DIM = 600,
         word_index = word_index)

# Fills empty rows with random number
    rnd <- runif(n=sum(is.na(embedding_matrix)), min = 0, max = .04)
    embedding_matrix[is.na(embedding_matrix)] <- rnd

# Test it
   table(is.na(embedding_matrix))

# See shape of embedding matrix and first row (needs to be zero, don't know why)
# https://github.com/rstudio/keras/issues/302)
   dim(embedding_matrix)
   embedding_matrix[1, ]


In [40]:

    hist(map_dbl(sequences, length))

# Shuffle data
    set.seed(8)

    indices <- sample(1:length(sequences))

    prop_train <- .82
    maxlen <-250

    train_indices <- 1: round(prop_train*length(sequences), 0)
    test_indices <-  (round(prop_train*length(sequences), 0)+1) : length(sequences)


    x_train <- pad_sequences(sequences[indices[train_indices]], maxlen = maxlen)
    x_test <- pad_sequences(sequences[indices[test_indices]], maxlen = maxlen)

    db_ys <- db_textos_splited[ , "id"] %>%
      as.data.frame() %>%
      set_names("id") %>%
      mutate(id = as.numeric(id)) %>%
      left_join( {db_bfi %>% select(id, O_rec:N_vlti_rec)}, by = "id" )


    y_train <- as.matrix(db_ys[indices[train_indices] , 2:6])
    y_test <-  as.matrix(db_ys[indices[test_indices] , 2:6])

    dim(x_train)
    dim(x_test)

    dim(y_train)
    dim(y_test)


[1] 9460    5

In [30]:
  max_words = 66128
    embedding_dim = 600

# bidirectional lstm model

   model <- keras_model_sequential() %>%
     layer_embedding(
         input_dim = max_words + 1,
         output_dim = embedding_dim ,
         weights = list(embedding_matrix),
         input_length = maxlen,
         trainable = FALSE ) %>%
    bidirectional(layer_lstm(units = 64,recurrent_dropout = 0.5, dropout =.5)) %>%
    layer_dense(units = 5,  kernel_regularizer = regularizer_l2(0.001))

    summary(model)


Model: "sequential_1"
________________________________________________________________________________
 Layer (type)                  Output Shape               Param #    Trainable  
 embedding_1 (Embedding)       (None, 250, 600)           39677400   N          
 bidirectional_1 (Bidirection  (None, 128)                340480     Y          
 al)                                                                            
 dense_1 (Dense)               (None, 5)                  645        Y          
Total params: 40018525 (152.66 MB)
Trainable params: 341125 (1.30 MB)
Non-trainable params: 39677400 (151.36 MB)
________________________________________________________________________________


In [ ]:

# for continouse y-variable

class_callback_loss_history <- new_callback_class(
    "loss_history",
    initialize = function() {
      self$batch_losses <- numeric()
    },
    on_batch_end = function(batch, logs) {
      # Append current batch's loss to the list
      self$batch_losses <- c(self$batch_losses, logs$get("loss"))

      # Plot the losses
      df <- data.frame(Batch = 1:length(self$batch_losses), Loss = self$batch_losses)
      print(
        ggplot(df, aes(x = Batch, y = Loss)) +
          geom_line(color = "blue") +
          labs(title = "Training Loss per Batch", x = "Batch", y = "Loss") +
          theme_minimal()
      )
    }
)

    model %>% compile(
        optimizer = "rmsprop",
        loss = "mse",
        metrics = c("mae")
    )


    history <- model %>% fit(
      x_train,
      y_train,
      epochs =9,
      batch_size = 100,
      validation_data = list(x_test, y_test),

      callbacks = list(class_callback_loss_history())
    )

    plot(history)


In [ ]:

    results <-  model %>% evaluate(x_test, y_test)
    predictions <-  model %>% predict(x_test)
    results

    cbind(predictions, y_test) %>%  corr.test()

    ggplot(
        data = data.frame(
            cbind(y_test, y_pred = predictions[ , 1])
            ),
        aes(x = y_test, y = y_pred) ) +
        geom_point( alpha = 1/2) +
        geom_smooth(method = "lm") +
        geom_smooth(color = "red")

   resp2[indices[test_indices], ] %>%
       cbind(predictions) %>%
       select(Código, Ma_measure, y_theta_z, predictions) %>%
       group_by(Código) %>%
       summarise_all(.funs = mean) %>%
       select(-Código) %>%
       ggplot(aes(x = Ma_measure, y = predictions) ) +
        geom_point( alpha = 1/2) +
        geom_smooth(method = "lm") +
        geom_smooth(color = "red")


    resp2[indices[test_indices], ] %>%
       cbind(predictions) %>%
       select(Código, Ma_measure, y_theta_z, predictions) %>%
       group_by(Código) %>%
       summarise_all(.funs = mean) %>%
       select(-Código) %>%
       corr.test()


    results <- model %>% evaluate(x_train, y_train)
    predictions <- model %>% predict(x_train)
    cor(predictions, y_train)

    ggplot(
        data = data.frame(
            cbind(y_train, y_pred = predictions[ , 1])
            ),
        aes(x = y_train, y = y_pred) ) +
        geom_point( alpha = 1/2) +
        geom_smooth(color = "orange") +
        geom_smooth(method = "lm")

